# Extract Time Series Data from MIA/KIA Data
John Bonfardeci<br/>
2021-02-02

In [27]:
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.appName('dpaa').getOrCreate()

In [28]:
spark

In [29]:
sql = SQLContext(spark.sparkContext)

In [49]:
data = spark.read.csv('./dpaa_icf_20210202.csv', header=True, inferSchema=True)
df = data.select(['individual_type', 'identified_date'])
sql.registerDataFrameAsTable(df, 'dpaa')

In [50]:
identified_count = sql.sql("""
SELECT 
    sum(identified) as IdentifiedCount
    , MONTH(identified_date) as Mn
    , YEAR(identified_date) as Yr
    , concat(MONTH(identified_date), '/1/', YEAR(identified_date)) as Monthly
FROM (
    SELECT
        case when 
            individual_type in ('Accounted for (1941-1973)', 'DoD Identification', 'Non DPAA Resolved') then 1
            else 0
        end as identified
        , cast(identified_date as Date) as identified_date
    FROM 
        dpaa
    WHERE
        identified_date != 'NULL' AND identified_date IS NOT NULL
) as t
WHERE identified_date > '2009-12-31'
GROUP BY
    MONTH(identified_date)
    , YEAR(identified_date)
ORDER BY Yr, Mn
""")
identified_count.show()

+---------------+---+----+---------+
|IdentifiedCount| Mn|  Yr|  Monthly|
+---------------+---+----+---------+
|              3|  1|2010| 1/1/2010|
|             10|  2|2010| 2/1/2010|
|             15|  3|2010| 3/1/2010|
|              4|  4|2010| 4/1/2010|
|              6|  5|2010| 5/1/2010|
|              2|  6|2010| 6/1/2010|
|              9|  8|2010| 8/1/2010|
|             12|  9|2010| 9/1/2010|
|              8| 10|2010|10/1/2010|
|             10| 11|2010|11/1/2010|
|             15|  1|2011| 1/1/2011|
|              6|  2|2011| 2/1/2011|
|              1|  3|2011| 3/1/2011|
|              3|  4|2011| 4/1/2011|
|              8|  5|2011| 5/1/2011|
|              4|  6|2011| 6/1/2011|
|              1|  7|2011| 7/1/2011|
|             11|  8|2011| 8/1/2011|
|             13|  9|2011| 9/1/2011|
|              4| 10|2011|10/1/2011|
+---------------+---+----+---------+
only showing top 20 rows



In [51]:
identified_count.select(['IdentifiedCount', 'Monthly']) \
    .repartition(1) \
    .write.csv('./dpaa_identified_counts', header=True)

In [20]:
schema = {}
count_data = spark.read.csv('./dpaa_icf_20210202_counts.csv', inferSchema=True, header=True)
count_data.show()

+---------------+----------+
|IdentifiedCount|   Monthly|
+---------------+----------+
|              3|2010-01-01|
|             10|2010-02-01|
|             15|2010-03-01|
|              4|2010-04-01|
|              6|2010-05-01|
|              2|2010-06-01|
|              0|2010-07-01|
|              9|2010-08-01|
|             12|2010-09-01|
|              8|2010-10-01|
|             10|2010-11-01|
|              0|2010-12-01|
|             15|2011-01-01|
|              6|2011-02-01|
|              1|2011-03-01|
|              3|2011-04-01|
|              8|2011-05-01|
|              4|2011-06-01|
|              1|2011-07-01|
|             11|2011-08-01|
+---------------+----------+
only showing top 20 rows



In [5]:
sql.registerDataFrameAsTable(count_data, 'count_data')

In [26]:
monthly_avg = sql.sql("""
SELECT AVG(IdentifiedCount) as MonthlyAvg FROM count_data WHERE YEAR(Monthly) >= 2015
""")
monthly_avg.show()

+------------------+
|        MonthlyAvg|
+------------------+
|13.865671641791044|
+------------------+



In [34]:
monthly_avg = sql.sql("""
SELECT ROUND(AVG(IdentifiedCount), 2) as Mean, MONTH(Monthly) as Mnth 
FROM count_data 
WHERE YEAR(Monthly) >= 2015 
GROUP BY MONTH(Monthly) 
ORDER BY Mnth
""")
monthly_avg.show()

+-----+----+
| Mean|Mnth|
+-----+----+
| 7.33|   1|
| 7.67|   2|
| 8.67|   3|
|13.83|   4|
|10.83|   5|
|15.67|   6|
| 14.0|   7|
| 31.4|   8|
| 31.6|   9|
|  5.6|  10|
|  8.4|  11|
| 15.2|  12|
+-----+----+



In [35]:
monthly_avg.repartition(1).write.csv('./monthly_means', header=True)

In [48]:
yearly_totals = sql.sql("""
SELECT SUM(IdentifiedCount) AS YrlyTotal, YEAR(Monthly) AS Yr
FROM count_data 
WHERE YEAR(Monthly) >= 2015 
GROUP BY YEAR(Monthly)
ORDER BY Yr
""")
yearly_totals.show()

+---------+----+
|YrlyTotal|  Yr|
+---------+----+
|      100|2015|
|      147|2016|
|      178|2017|
|      215|2018|
|      214|2019|
|       75|2020|
+---------+----+



In [45]:
yearly_totals.repartition(1).write.csv('./yearly_totals', header=True)

In [38]:
yearly_avg = sql.sql("""
SELECT AVG(YrlyTotal) as YrlyAvg FROM (
    SELECT SUM(IdentifiedCount) AS YrlyTotal FROM count_data WHERE YEAR(Monthly) >= 2015 GROUP BY YEAR(Monthly)
)
""")
yearly_avg.show()

+------------------+
|           YrlyAvg|
+------------------+
|154.83333333333334|
+------------------+



In [47]:
monthly_avg_yr = sql.sql("""
SELECT ROUND(AVG(IdentifiedCount), 2) AS MonthlyAvg, YEAR(Monthly) AS Yr
FROM count_data 
WHERE YEAR(Monthly) >= 2015 
GROUP BY YEAR(Monthly)
ORDER BY Yr
""")
monthly_avg_yr.show()

+----------+----+
|MonthlyAvg|  Yr|
+----------+----+
|      8.33|2015|
|     12.25|2016|
|     14.83|2017|
|     17.92|2018|
|     17.83|2019|
|     10.71|2020|
+----------+----+



In [41]:
monthly_avg_yr.repartition(1).write.csv('./monthly_means_yr', header=True)